# SPARC-4 mini-pipeline tools
## Quarter-wave polarimetry

This notebook shows an example for obtaining a polarimetric analysis of a set of observations using the SPARC4 instrument in polarimetric mode with a quarter-wave rotating plate. It takes as input a series of science imaging products obtained with the SPARC4 pipeline.

* Eder Martioli, LNA 16 Feb 2023

In [1]:
import os,sys
path_parent = os.path.dirname(os.getcwd())
# path_parent = "/Volumes/Samsung_T5/sparc4-pipeline/"
sys.path.append(path_parent) 
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import numpy as np
import glob

import sparc4_product_plots as s4plt
import sparc4_pipeline_lib as s4pipelib
import sparc4_utils as s4utils
import sparc4_params

### User inputs

In [2]:
# set night directory name
NIGHTDIR = '20221115'
# set SPARC4 channel
CHANNEL = 3  # 1, 2, 3 or 4
# set an object ID matching the ID in the image header keyword 'OBJECT'
OBJECTID = "NGC20241"
# whether or not to force reduction even if product already exists
FORCE = True

### Preparatory procedures
Below are some preparatory procedures for defining data directories and product file names.

In [4]:
# get SPARC4 pipeline parameters
p = sparc4_params.load_sparc4_parameters()
# set raw data root directory (where raw data will be searched)
p['ROOTDATADIR'] = "/Volumes/Samsung_T5/Data/SPARC4/comissioning_nov22/"
# set reduction directory name
p['ROOTREDUCEDIR'] = '/Volumes/Samsung_T5/Data/SPARC4/comissioning_nov22/reduced'
# if reduced dir doesn't exist create one
if not os.path.exists(p['ROOTREDUCEDIR']) :
    os.mkdir(p['ROOTREDUCEDIR'])
    
# identify files for reduction
p = s4utils.identify_files(p, NIGHTDIR, print_report=False)    

# set full paths for a given night and channel
data_dir = p['data_directories'][CHANNEL-1]
ch_reduce_dir = '{}/sparc4acs{}/'.format(p['ROOTREDUCEDIR'],CHANNEL)
reduce_dir = '{}/sparc4acs{}/{}/'.format(p['ROOTREDUCEDIR'],CHANNEL,NIGHTDIR)